In [42]:
def CompareRegressors(X_train, X_test, Y_train, Y_test): 
    import numpy as np
    import pandas as pd
    
    from sklearn.neural_network import MLPRegressor    
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.gaussian_process import GaussianProcessRegressor
    
    from sklearn.gaussian_process.kernels import RBF
    from sklearn.gaussian_process.kernels import ExpSineSquared
    from sklearn.gaussian_process.kernels import RationalQuadratic
    from sklearn.gaussian_process.kernels import ConstantKernel
    from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

        
   
    regression_models=[]
    
    r_score=0
    
    ############################## MLPR ###################################################
    for activation_function in ("identity", "logistic", "tanh", "relu"):
        
        reg = MLPRegressor(random_state=1, max_iter=500,activation=activation_function).fit(X_train, Y_train)
        regression_models.append(("MLP",activation_function,reg,reg.score(X_test, Y_test)))
    
    
    ########################  "BOOSTED REGRESSION TREES"  ###############################
    for n_estimators in (5,10,20,50,100,200,300,400,500):
    
        reg = GradientBoostingRegressor(n_estimators=n_estimators,
                                    max_depth=3,
                                    learning_rate=0.1,
                                    min_samples_split=3)
        reg.fit(X_train, Y_train)
        regression_models.append(("BOOSTED REGRESSION TREES",n_estimators, reg, reg.score(X_test, Y_test)))
        
        
    ##################"Gaussian Process Regressor"############################
    kernel_types=[]
    kernel1 = 1.0 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 10.0))
    kernel2= 1.0 * RationalQuadratic(length_scale=1.0, alpha=0.1, alpha_bounds=(1e-5, 1e15))
    #kernel3= 1.0 * ExpSineSquared(length_scale=1.0,periodicity=3.0,length_scale_bounds=(0.1, 10.0),periodicity_bounds=(1.0, 10.0),)
    kernel4 = DotProduct() + WhiteKernel()
    
    kernel_types.append(kernel1)
    kernel_types.append(kernel2)
    #kernel_types.append(kernel3)
    kernel_types.append(kernel4)
    
    for kernel in kernel_types:
    
        reg = GaussianProcessRegressor(kernel=kernel, random_state=0)
        reg.fit(X_train, Y_train)
        # R^2 score for the model (on the test data)
        regression_models.append(("Gaussian Process Regressor",kernel, reg, reg.score(X_test, Y_test)))
        
    #print(len(regression_models))
        
    ########################now, save the best model ##############################
    
    import joblib
    max_r_score=0
    for i in range(len(regression_models)):
        print(regression_models[i][3])
        model_r=float(regression_models[i][3])
        if(model_r > max_r_score):
            max_r_score=model_r
        
        
    for k in range(len(regression_models)): 
        if(regression_models[k][3] == max_r_score):
            # this is our best model
            print(regression_models[k])
            joblib.dump(regression_models[k][2],"best_regr_model.jl")
            break
    print("best r square score achieved: ", max_r_score) 
    print(regression_models[k])
    return regression_models

In [43]:
import numpy as np

import pandas as pd


df = pd.read_excel('Query_11.xlsx')

df=df.head(1000)

labels = np.array(df['yield_Value'])

features= df.drop(['year_obs',  'State_ANSI', 'County_ANSI', 'yield_Value'], axis = 1)# Remove the Crop Yield from the features
#features= df.drop(['year_obs',  'State_ANSI', 'County_ANSI', 'yield_Value', 'b1','b2','b3','b4','b5','b6','b7','tmin','tmax'], axis = 1)#

#features= df.drop(['year_obs',  'State_ANSI', 'County_ANSI', 'yield_Value', 'b1','b2','b3','b4','b5','b6','b7'], axis = 1)#
#features= df.drop(['year_obs',  'State_ANSI', 'County_ANSI', 'yield_Value', 'tmin','tmax','prcp','heat_cold_wave_indx','drought_indx'], axis = 1)
feature_list = list(features.columns)# Saving feature names for later use

features = np.array(features)# Convert features to numpy array


np.unique(labels)

array([126, 127, 131, 141, 144, 145, 147, 150, 152, 155, 159, 168, 170,
       171, 173, 175, 177, 181, 183, 185, 192], dtype=int64)

In [44]:
df.head()

b1           b2          b3           b4           b5  \
0  1119.173462  2241.181641  573.418701   914.979065  3128.707764   
1   579.056091  3585.113037  314.819855   708.317017  3534.294189   
2  1034.131714  3708.768555  542.762329  1031.504272  4131.856445   
3   926.238098  3318.897217  486.446289   924.007141  3715.292236   
4   988.537048  2786.892578  570.810364   928.292786  3230.314209   

            b6           b7       tmin       tmax      prcp  \
0  3247.370361  2215.685791   2.659751  11.115650  3.476146   
1  2493.193359  1221.297485  16.896063  29.698597  5.147169   
2  3553.105469  2450.981445  15.156509  28.380976  0.000000   
3  3262.375488  2188.172119  14.386539  25.148739  5.875622   
4  2927.056152  2054.672607  11.171411  21.794228  2.820405   

   heat_cold_wave_indx  drought_indx      NDVI       EVI      NDWI  year_obs  \
0                 0.00           1.0  0.329689  0.565295 -0.169698      2003   
1                 0.00           1.0  0.716725  1.506106  0.003644      2003   
2                 0.00           1.0  0.563903  1.096366 -0.055305      2003   
3                 0.00           0.0  0.562693  1.090795 -0.057005      2003   
4                 0.05           0.0  0.480286  0.899202 -0.078630      2003   

   State_ANSI  County_ANSI  yield_Value  
0          17            1          171  
1          17            1          171  
2          17            1          171  
3          17            1          171  
4          17            1          171

In [45]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.30, random_state = 42)# Split the data into 70% training and 30% testing sets


In [46]:
import matplotlib.pyplot as plt

In [47]:
models= CompareRegressors(train_features, test_features, train_labels, test_labels)

C:\Users\karm_ch\AppData\Local\Continuum\anaconda3\envs\gdal_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\karm_ch\AppData\Local\Continuum\anaconda3\envs\gdal_env\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


-1.9562801850890055
-2.3310458779918353
-4.253351860095781
-1.2452708189803143
0.16540240097854794
0.23280890626941952
0.3179024879912892
0.4219317053651499
0.46945947214600714
0.4861505840109568
0.4902897638770728
0.4892837128011976
0.4806205846553473
-76.52860315001672
0.3844688786538685
0.10075799937298259
('BOOSTED REGRESSION TREES', 300, GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=3,
                          min_weight_fraction_leaf=0.0, n_estimators=300,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_st

In [ ]:
models